## 1 Objectif 

L’objectif de ce projet est de programmer un modèle de langage neuronal à l’aide d’un perceptron multi-couches. Ce modèle de langage prend en entrée les plongements de k mots consécutifs et produit en sortie une distribution de probabilité sur l’ensemble du vocabulaire. Contrairement au projet précédent, nous ne ferons pas le calcul du gradient à la main, mais utiliserons la librairie pytorch et les outils de dérivation automatique qu’elle propose.


## 2 Leclassifieur 
Le cœur du modèle est constitué d’un perceptron multicouche \(C\) composé d’une couche d’entrée \(x\), d’une couche cachée \(h\) et d’une couche de sortie \(y\). Les paramètres du modèle sont regroupés dans deux matrices : \(W\) et \(U\) et deux vecteurs : \(b_1\) et \(b_2\). \(W\) et \(b_1\) permettent de calculer les valeurs de la couche cachée à partir de la couche d’entrée tandis que \(U\) et \(b_2\) permettent de calculer les valeurs de la couche de sortie à partir de la couche cachée. Étant donné une séquence de mots \(m_1, \ldots, m_n\), le classifieur \(C\) prend en entrée les plongements des \(k\) mots \(m_{i-k}, \ldots, m_{i-1}\) et produit en sortie une distribution de probabilité sur l’ensemble des mots du vocabulaire \(V\). On considère que les plongements de chaque mot sont de dimension \(d\). La taille de la couche d’entrée, notée \(x\), vaut par conséquent \(d_x = k \times d\) et la couche de sortie a pour dimension \(|V|\), la taille du vocabulaire. La dimension de la couche cachée est arbitraire, on la notera \(d_h\). Les dimensions des deux matrices \(W\) et \(U\) sont donc \(d_x \times d_h\) pour \(W\) et \(d_h \times |V|\) pour \(U\).

Le calcul de la couche cachée peut être décomposé en deux étapes :
- une étape linéaire : \(h' = xW + b_1\)
- suivie d’une étape non linéaire : \(h = \text{ReLU}(h')\)

De même, le calcul de la couche de sortie peut aussi être décomposé en deux étapes :
- une étape linéaire : \(y' = hU + b_2\)
- suivie d’une étape non linéaire : \(y = \text{Softmax}(y')\)

La fonction \(y = \text{softmax}(y')\) permet de transformer un vecteur de valeurs réelles \(y'\) en un vecteur de probabilités \(y\). Chaque composante \(y_i\) de \(y\) est calculée de la manière suivante :

\[ y_i = \frac{\exp(y'_i)}{\sum_{j=1}^{|V|} \exp(y'_j)} \]

Autrement dit, la fonction \(\exp(\cdot)\) rend la valeur de \(y'_i\) positive, et ensuite chaque valeur est normalisée par la somme de toutes les valeurs du vecteur \(y'_j\). Cela garantit que \(y_i\) est bien une probabilité car \(\sum_{i=1}^{|V|} y_i = 1\).

In [1]:
import torch

In [2]:
import torch
import numpy as np
import torch
import sys

class Vocab:
    def __init__(self, **kwargs):

        self.dico_voca = {}
        self.word_array = []
        if "emb_filename" in kwargs :
            with open(kwargs["emb_filename"],'r') as fi:
                ligne = fi.readline()
                ligne = ligne.strip()
                (self.vocab_size, self.emb_dim) = map(int,ligne.split(" "))
                self.matrice = torch.zeros((self.vocab_size, self.emb_dim))
                indice_mot = 0
        
                ligne = fi.readline()
                ligne = ligne.strip()
                while ligne != '': 
                    splitted_ligne = ligne.split()
                    self.dico_voca[splitted_ligne[0]] = indice_mot
                    self.word_array.append(splitted_ligne[0])
                    for i in range(1,len(splitted_ligne)):
                        self.matrice[indice_mot, i-1] = float(splitted_ligne[i])
                    indice_mot += 1
                    ligne = fi.readline()
                    ligne = ligne.strip()
        else:
            fichier_corpus = kwargs["corpus_filename"]
            self.emb_dim = kwargs["emb_dim"]
            nb_tokens = 0
            with open(fichier_corpus,'r') as fi:
                for line in fi:
                    line = line.rstrip()
                    tokens = line.split(" ")
                    for token in tokens:
                        if token not in self.dico_voca :
                            self.word_array.append(token)
                            self.dico_voca[token] = nb_tokens
                            nb_tokens += 1
            self.vocab_size = nb_tokens
            print("vocab size =", self.vocab_size, "emb_dim =", self.emb_dim)
            self.matrice = torch.zeros((self.vocab_size, self.emb_dim))

    def get_word_index(self, mot):
        if not mot in self.dico_voca:
            return None
        return self.dico_voca[mot]
                
    def get_word_index2(self, mot):
        if not mot in self.dico_voca:
            return self.dico_voca['<unk>']
        return self.dico_voca[mot]
                
    def get_emb(self, mot):
        if not mot in self.dico_voca:
            return None
        return  self.matrice[self.dico_voca[mot]]
    
    def get_emb_torch(self, indice_mot):
        # OPTIMISATION: no verificaiton allows to get embeddings a bit faster
        #if indice_mot < 0 or indice_mot >= self.matrice.shape()[0]: # not valid index
        #    return None
        #return self.matrice[indice_mot]
        return self.matrice[indice_mot]
        
    def get_one_hot(self, mot):
        vect = torch.zeros(len(self.dico_voca))
        vect[self.dico_voca[mot]] = 1
        return vect

    def get_word(self, index):
        if index < len(self.word_array):
            return self.word_array[index]
        else:
            return None



In [5]:
Vocab(emb_filename="../data/embeddings-word2vecofficial.train.unk5.txt")
W, U = 



1. Ecrire la fonction de pr´ eparation des donn´ ees qui prend en entr´ ee un f ichier de texte T ainsi qu’un dictionnaire associant `a tout mot du vo3cabulaire, un indice1. Cette fonction produit en sortie une liste dont les ´el´ ements sont des listes de k + 1 mots cons´ ecutifs de T repr´ esent´ es par leur indices. Par exemple, si le corpus se pr´ esente de la fa¸con suivante a b c d e f et que le mot a correspond `a l’indice 1, le mot b correspond `a l’indice 2 . . .et que k vaut 3, les listes X et Y se pr´ esenteront sous la forme suivante : X = [[1,2,3,4],[2,3,4,5],[3,4,5,6]]

In [ ]:
def prepare_data(file_path, vocab, k=3):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read().split()
    
    data = []
    for i in range(len(text) - k):
        sequence = [Vocab.get_word_index(word) for word in text[i:i+k+1]]
        data.append(sequence)
    return data

prepare_data("Le_co", Vo, k=3)


In [7]:
var = torch.tensor([[1,0,3], [0,1,3]])
var.size(0)

2

In [3]:
def modele(k, d, h, V):
    """modélisation du perceptron possédant une couche d'entrée de taille k*d, une couche cachée de taille h et une couche de sortie de taille la dimension de V, les matrices de poids sont  W et U"""
    
    W, U = Vocab(emb_filename="../embeddings.txt")
    

    return W, U

In [ ]:
def extraction(plongements):
    """extraction des plongements de taille d à partir de la matrice de plongements de taille k*d"""
    


In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU available')
else:
    device = torch.device('cpu')

def forward(W, U, x):
    """propagation avant du perceptron"""
    h = torch.mm(W, x)
    h = torch.relu(h)
    y = torch.mm(U, h)
    y = torch.softmax(y, 0)
    return y

def loss(y, y_pred):
    """calcul de la fonction de perte"""
    return -torch.sum(y*torch.log(y_pred))

def backward(W, U, x, y, y_pred):
    """rétropropagation du gradient"""
    dL_dy = y_pred - y
    dL_dU = torch.mm(dL_dy, h.t())
    dL_dh = torch.mm(U.t(), dL_dy)
    dL_dh[h <= 0] = 0
    dL_dW = torch.mm(dL_dh, x.t())
    return dL_dW, dL_dU

def update(W, U, dL_dW, dL_dU, lr):
    """mise à jour des poids"""
    W -= lr*dL_dW
    U -= lr*dL_dU
    return W, U

def train(W, U, x, y, lr):
    """entraînement du perceptron"""
    y_pred = forward(W, U, x)
    l = loss(y, y_pred)
    dL_dW, dL_dU = backward(W, U, x, y, y_pred)
    W, U = update(W, U, dL_dW, dL_dU, lr)
    return l

def test(W, U, x, y):
    """test du perceptron"""
    y_pred = forward(W, U, x)
    l = loss(y, y_pred)
    return l

def main():
    W, U = Vocab(emb_filename="../embeddings.txt")
    W = W.to(device)
    U = U.to(device)
    W.requires_grad = True
    U.requires_grad = True
    lr = 0.01
    for i in range(100):
        l = train(W, U, x, y, lr)
        print("Train Epoch: "+i+" Loss: "+l)
        
    l = test(W, U, x, y)
    print(l)



GPU available


NameError: name 'k' is not defined

In [ ]:
yf = (np.exp(y))/np.sum(np.exp(y))

In [ ]:


def modèle(plongements="plongements"):

    """Fonction qui prend en entrée le fichier de plongements (k mots de d plongements) et qui produit la distribution de probabilité sur l'ensemble des mots du vocabulaire"""
    W, U = modele(k, d, h, V)
    